### Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
current_palette = sns.color_palette()
import warnings
warnings.filterwarnings('ignore')
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from IPython.display import clear_output
sns.set_style('darkgrid')
np.random.seed(0)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import retrieve as rt

def plot_feature_importances(model):
    n_features = data_train.shape[1]
    plt.figure(figsize=(8,34))
    plt.barh(range(n_features), model.feature_importances_, align='center') 
    plt.yticks(np.arange(n_features), data_train.columns.values) 
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')


### Obtain

In [2]:
total = rt.merged_data()
clean = rt.clean_data(total)
income = rt.income_data(total)
poverty = rt.poverty_data(total)
race = rt.race_data(total)
population = rt.sex_age_data(total)

In [3]:
for i in clean.columns:
    if clean[i].isna().sum() > 0:
        print(i)

In [4]:
data = clean
multiplier = 0

In [5]:
target_column='ratio_to_max_payment'

y= data.ratio_to_max_payment >= multiplier*data.ratio_to_max_payment.std()+data.ratio_to_max_payment.mean()
X = data.drop(columns=[target_column])
data_train, data_test, target_train, target_test = train_test_split(X, y, 
                                                                    test_size = 0.25)


forest = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=5)
forest.fit(data_train,target_train)
df_feature_importance = pd.Series(data = forest.feature_importances_, index= X.columns).sort_values(ascending=False)
df_feature_importance[:12]

Percent_Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_Some_other_race    0.049260
with_cc                                                                                               0.043190
with_mcc                                                                                              0.039730
Percent_Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_Asian              0.032701
CARDIAC ARRHYTHMIA & CONDUCTION DISORDERS                                                             0.031912
Provider_Zip_Code                                                                                     0.031526
Percent_RACE_Total_population_One_race_Asian                                                          0.030386
Percent_HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_White_alone               0.027455
Percent_RACE_Total_population_One_race_Some_other_race                                                0.026013
P

In [6]:
forest.score(data_train,target_train), forest.score(data_test,target_test)

(0.6728307254623044, 0.6732647009185814)

In [22]:
model_clean, feat = rt.random_forest(clean, multiplier=1)

Percent_HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_Asian_alone                           0.042577
Percent_RACE_Total_population_One_race_Asian_Chinese                                                              0.038952
Percent_HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_White_alone                           0.035836
Total_housing_units                                                                                               0.033802
Percent_RACE_Total_population_One_race_Asian                                                                      0.033015
Percent_SEX_AND_AGE_Total_population_18_years_and_over.1                                                          0.030816
Percent_Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_Asian                          0.029106
SEX_AND_AGE_Total_population                                                                                      0.027860
Percent_Race_alo

In [8]:
data = income

In [9]:
target_column='ratio_to_max_payment'
y = pd.cut(data[target_column],3).astype(str)
y= data.ratio_to_max_payment >= multiplier*data.ratio_to_max_payment.std()+data.ratio_to_max_payment.mean()
X = data.drop(columns=[target_column])
data_train, data_test, target_train, target_test = train_test_split(X, y, 
                                                                    test_size = 0.25)


forest = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=5)
forest.fit(data_train,target_train)
df_feature_importance = pd.Series(data = forest.feature_importances_, index= X.columns).sort_values(ascending=False)
df_feature_importance[:12]

CARDIAC ARRHYTHMIA & CONDUCTION DISORDERS       0.101306
Households_Total                                0.078203
Families_Total                                  0.077163
SEX_AND_AGE_Total_population                    0.059640
Nonfamily_households_Median_income_(dollars)    0.051610
LAPAROSCOPIC CHOLECYSTECTOMY                    0.035822
Households_$150,000_to_$199,999                 0.033801
BRONCHITIS & ASTHMA                             0.031283
Families_$50,000_to_$74,999                     0.025316
Households_$200,000_or_more                     0.024069
Households_$25,000_to_$34,999                   0.023999
Households_$50,000_to_$74,999                   0.023413
dtype: float64

In [10]:
forest.score(data_train,target_train), forest.score(data_test,target_test)

(0.6705965555207304, 0.6764954070930878)

In [11]:
data = poverty

In [12]:
target_column='ratio_to_max_payment'
y = pd.cut(data[target_column],3).astype(str)
y= data.ratio_to_max_payment >= multiplier*data.ratio_to_max_payment.std()+data.ratio_to_max_payment.mean()
X = data.drop(columns=[target_column])
data_train, data_test, target_train, target_test = train_test_split(X, y, 
                                                                    test_size = 0.25)


forest = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=5)
forest.fit(data_train,target_train)
df_feature_importance = pd.Series(data = forest.feature_importances_, index= X.columns).sort_values(ascending=False)
df_feature_importance[:12]

SEX_AND_AGE_Total_population                                                                                                                  0.183912
CARDIAC ARRHYTHMIA & CONDUCTION DISORDERS                                                                                                     0.105649
BRONCHITIS & ASTHMA                                                                                                                           0.062106
LAPAROSCOPIC CHOLECYSTECTOMY                                                                                                                  0.061264
Percent_below_poverty_level_UNRELATED_INDIVIDUALS_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED_Did_not_work                                          0.061028
Percent_below_poverty_level_Population_for_whom_poverty_status_is_determined_AGE_65_years_and_over                                            0.048847
PULMONARY EDEMA & RESPIRATORY FAILURE                                                         

In [13]:
forest.score(data_train,target_train), forest.score(data_test,target_test)

(0.6856731843350934, 0.6883815126455626)

In [14]:
data = race

In [15]:
target_column='ratio_to_max_payment'
y = pd.cut(data[target_column],3).astype(str)
y= data.ratio_to_max_payment >= multiplier*data.ratio_to_max_payment.std()+data.ratio_to_max_payment.mean()
X = data.drop(columns=[target_column])
data_train, data_test, target_train, target_test = train_test_split(X, y, 
                                                                    test_size = 0.25)


forest = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=5)
forest.fit(data_train,target_train)
df_feature_importance = pd.Series(data = forest.feature_importances_, index= X.columns).sort_values(ascending=False)
df_feature_importance[:12]

Percent_Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_Some_other_race                0.068125
Percent_RACE_Total_population_One_race_Asian_Chinese                                                              0.063463
Percent_HISPANIC_OR_LATINO_AND_RACE_Total_population_Not_Hispanic_or_Latino_White_alone                           0.057722
CARDIAC ARRHYTHMIA & CONDUCTION DISORDERS                                                                         0.057490
Percent_Race_alone_or_in_combination_with_one_or_more_other_races_Total_population_Asian                          0.054064
Percent_RACE_Total_population_One_race_Some_other_race                                                            0.051739
Percent_RACE_Total_population_One_race_Asian                                                                      0.047069
Percent_HISPANIC_OR_LATINO_AND_RACE_Total_population_Hispanic_or_Latino_(of_any_race)_Other_Hispanic_or_Latino    0.037650
LAPAROSCOPIC CHO

In [16]:
forest.score(data_train,target_train), forest.score(data_test,target_test)

(0.6689008366082407, 0.6663210936181498)

In [17]:
data = population

In [18]:
target_column='ratio_to_max_payment'
y = pd.cut(data[target_column],3).astype(str)
y= data.ratio_to_max_payment >= multiplier*data.ratio_to_max_payment.std()+data.ratio_to_max_payment.mean()
X = data.drop(columns=[target_column])
data_train, data_test, target_train, target_test = train_test_split(X, y, 
                                                                    test_size = 0.25)


forest = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=5)
forest.fit(data_train,target_train)
df_feature_importance = pd.Series(data = forest.feature_importances_, index= X.columns).sort_values(ascending=False)
df_feature_importance[:12]

CARDIAC ARRHYTHMIA & CONDUCTION DISORDERS                               0.100986
SEX_AND_AGE_Total_population                                            0.084360
Percent_CITIZEN_VOTING_AGE_POPULATION_Citizen_18_and_over_population    0.073109
Percent_SEX_AND_AGE_Total_population_18_years_and_over.1                0.068742
Percent_SEX_AND_AGE_Total_population_65_years_and_over.1                0.067971
Percent_SEX_AND_AGE_Total_population_65_to_74_years                     0.049434
LAPAROSCOPIC CHOLECYSTECTOMY                                            0.044777
BRONCHITIS & ASTHMA                                                     0.037852
Percent_SEX_AND_AGE_Total_population_25_to_34_years                     0.034073
Percent_SEX_AND_AGE_Total_population_75_to_84_years                     0.030986
PULMONARY EDEMA & RESPIRATORY FAILURE                                   0.029064
MISC DISORDERS OF NUTRITION,METABOLISM,FLUIDS/ELECTROLYTES              0.025548
dtype: float64

In [19]:

forest.score(data_train,target_train), forest.score(data_test,target_test)

(0.662166180453424, 0.6600766689972756)